In [76]:
import pandas as pd
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
data = pd.read_csv("player_regular_season.csv")

In [77]:
data['fullName'] = data['firstname'].str.lower() + data['lastname'].str.lower()
data = data.drop(["firstname",'lastname', 'team'], axis=1).dropna()

In [78]:
#make %
data['fg_p'] = data['fgm']/data['fga']
data['ft_p'] = data['ftm']/data['fta']
data['tp_p'] = data['tpm']/data['tpa']
data = data.drop(['fgm', 'fga', 'ftm', 'fta', 'tpm', 'tpa'], axis=1).fillna(0, )

In [79]:
data.head()

,ilkid,year,leag,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fullName,fg_p,ft_p,tp_p
0,ABRAMJO01,1946,N,47,0,527,0,0,0,35,0,0,0.0,161,johnabramovic,0.242206,0.691011,0.0
1,AUBUCCH01,1946,N,30,0,65,0,0,0,20,0,0,0.0,46,chetaubuchon,0.252747,0.542857,0.0
2,BAKERNO01,1946,N,4,0,0,0,0,0,0,0,0,0.0,0,normbaker,0.000000,0.000000,0.0
3,BALTIHE01,1946,N,58,0,138,0,0,0,16,0,0,0.0,98,herschelbaltimore,0.201521,0.463768,0.0
4,BARRJO01,1946,N,58,0,295,0,0,0,54,0,0,0.0,164,johnbarr,0.283105,0.594937,0.0


In [80]:
player_years = {}
#make a dict of assosiation of the id with fullname
id_fullName = {}
for index, row in data.iterrows():
    if row['ilkid'] in player_years:
        player_years[row['ilkid']].append(row['year'])
    else:
        player_years[row['ilkid']] = [row['year']]
    id_fullName[row['ilkid']] = row['fullName']

In [81]:
training_players = [player for player in player_years.keys() if len(player_years[player]) >= 10]
new_players = [player for player in player_years.keys() if (len(player_years[player]) == 1 and 2004 in player_years[player]) or (len(player_years[player]) == 2 and (2003 in player_years[player] and 2004 in player_years[player]))]

In [82]:
data_index_full = data.set_index(['ilkid'])

In [83]:
#separating the data set

training = data_index_full.loc[training_players]
test = data_index_full.loc[new_players]


In [84]:
training_data = []
for play in training_players:
    training_data.extend(data[(data['ilkid'] == play) & (data['year'] == player_years[play][0])].values)

training_data = pd.DataFrame(training_data, columns=data.columns).set_index('ilkid')
training_data_clean = []
#get the mediam of players that played more than one team in the same year
for play in training_players:
    if len(training_data.loc[play]) != 17:
        aux = training_data.loc[play]
        year = aux.loc[:,['year']] # save the year
        aux = aux.drop(['year'], axis=1).sum() #delete the year to not count in the sum
        aux.name = play
        aux['year'] = year
        training_data_clean.append(aux)
    else:
        training_data_clean.append(training_data.loc[play])
training_data_clean = pd.DataFrame(training_data_clean)
training_data_clean.head()

,year,leag,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fullName,fg_p,ft_p,tp_p
MAHNKJO01,1946,N,60,0,557,0,0,0,60,0,0,0.0,181,johnmahnken,0.254566,0.680982,0.0
NOSTRGE01,year ilkid NOSTRGE01 194...,NNN,122,0,964,0,0,0,62,0,0,0.0,290,georgenostrandgeorgenostrandgeorgenostrand,0.911687,1.356754,0.0
SIMMOCO01,1946,N,60,0,620,0,0,0,62,0,0,0.0,130,conniesimmons,0.320312,0.677249,0.0
ZASLOMA01,1946,N,61,0,877,0,0,0,40,0,0,0.0,121,maxzaslofsky,0.329412,0.737410,0.0
BRAUNCA01,1947,N,47,0,671,0,0,0,61,0,0,0.0,102,carlbraun,0.323185,0.650273,0.0


In [85]:
training_data_clean.shape

(746, 17)

In [86]:
test_data = []
for play in new_players:
    test_data.extend(data[(data['ilkid'] == play) & (data['year'] == player_years[play][0])].values)
coluns_X = ['gp', 'minutes', 'pts', 'oreb', 'dreb', 'reb', 'asts', 'stl', 'blk', 'turnover', 'pf', 'fg_p', 'ft_p', 'tp_p']
test_data = pd.DataFrame(test_data, columns=data.columns).set_index('ilkid')
test_data.head()

,year,leag,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fullName,fg_p,ft_p,tp_p
ilkid,,,,,,,,,,,,,,,,,
ANTHOCA01,2003,N,82,2995,1725,183,315,498,227,97,41,247.0,225,carmeloanthony,0.425939,0.777143,0.322430
BANKSMA01,2003,N,81,1385,480,30,103,133,175,88,13,125.0,171,marcusbanks,0.400452,0.755725,0.313953
BARBOLE01,2003,N,70,1500,550,23,100,123,165,93,7,120.0,184,leandrinhobarbosa,0.446809,0.770492,0.395238
BARNEMA02,2003,N,38,719,171,53,98,151,48,27,3,44.0,76,mattbarnes,0.456522,0.704918,0.153846
BLAKEST01,2003,N,75,1391,444,18,99,117,209,57,7,128.0,100,steveblake,0.385749,0.820896,0.371287


In [87]:
X_COLUMNS = training_data.columns[2:-1]
X = training_data_clean[coluns_X]
Y = np.arange(X.shape[0])

In [88]:
test = test_data[coluns_X]

In [103]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
clf = clf.fit(X, Y)

nearts = clf.kneighbors(test, n_neighbors=5)

In [133]:
#max(aux)

In [104]:
def find_key_by_index(_dict, search_num):
    for player, num in _dict.items():
        if num == search_num:
            return player

In [105]:
dic_play_num = {}
for num, player in enumerate(training_data_clean.index.tolist()):
    dic_play_num[player] = num



In [106]:
data.head()

,ilkid,year,leag,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fullName,fg_p,ft_p,tp_p
0,ABRAMJO01,1946,N,47,0,527,0,0,0,35,0,0,0.0,161,johnabramovic,0.242206,0.691011,0.0
1,AUBUCCH01,1946,N,30,0,65,0,0,0,20,0,0,0.0,46,chetaubuchon,0.252747,0.542857,0.0
2,BAKERNO01,1946,N,4,0,0,0,0,0,0,0,0,0.0,0,normbaker,0.000000,0.000000,0.0
3,BALTIHE01,1946,N,58,0,138,0,0,0,16,0,0,0.0,98,herschelbaltimore,0.201521,0.463768,0.0
4,BARRJO01,1946,N,58,0,295,0,0,0,54,0,0,0.0,164,johnbarr,0.283105,0.594937,0.0


In [107]:
id_fullName

{'ABRAMJO01': 'johnabramovic',
 'AUBUCCH01': 'chetaubuchon',
 'BAKERNO01': 'normbaker',
 'BALTIHE01': 'herschelbaltimore',
 'BARRJO01': 'johnbarr',
 'BAUMHFR01': 'frankiebaumholtz',
 'BECKEMO01': 'moebecker',
 'BEENDHA01': 'hankbeenders',
 'BIASAHA01': 'hankbiasatti',
 'BRIGHAL01': 'albrightman',
 'BRINDAU01': 'audleybrindley',
 'BROWNHA01': 'haroldbrown',
 'BROWNLE01': 'leonbrown',
 'BYRNETO01': 'tommybyrnes',
 'BYTZUMI01': 'mikebytzura',
 'CALLATO01': 'tomcallahan',
 'CALVEER01': 'erniecalverley',
 'CARLICH01': 'chetcarlisle',
 'CARLSDO01': 'doncarlson',
 'CLUGGMA01': 'marioncluggish',
 'CONNOCH01': 'chuckconnors',
 'CORLEKE01': 'kencorley',
 'CRISLHA01': 'halcrisler',
 'CUREAR01': 'armandcure',
 'DALLMHO01': 'howiedallmar',
 'DAVISAU01': 'aubreydavis',
 'DAVISBI01': 'billdavis',
 'DEHNEHE01': 'henrydehnert',
 'DILLEBO01': 'bobdille',
 'DOLLBO01': 'bobdoll',
 'DUFFYBO01': 'bobduffy',
 'ELIASDO01': 'doneliason',
 'ENDRENE01': 'nedendress',
 'FABELJO01': 'joefabel',
 'FAUGHBO01': 'bobf

In [125]:
from statistics import mean
data_name_index = data.set_index('fullName')
play_with_ass = {}
play_smaller_dis  = 0

for play, near, dis in zip(new_players, nearts[1], nearts[0]):
    #get the mean of distance
    dis_mean = mean(dis)
    if play_smaller_dis < dis_mean:
        play_smaller_dis = dis_mean
        play_smaller_dis_name = id_fullName[play]

    #just get players with dist less then 20
  
    if dis_mean > 150:
        near_players = [id_fullName[find_key_by_index(dic_play_num, i)] for i in near]
        #for near player get the init statics and the mean of all the years
        near_players_data = {}
        for near_player in near_players:
            all_data = data_name_index.loc[near_player]
            aux = all_data[all_data['year'] == player_years[find_key_by_index(id_fullName, near_player)][0]]
            final = pd.DataFrame(aux.drop(['year', 'leag', 'gp', 'ilkid'], axis=1).mean())
            save = aux[['year', 'leag', 'gp', 'ilkid']].iloc[0, :]
            first_year = pd.concat([final,save], axis=0)            
            print(first_year)
            
            data_mean = all_data.reset_index().drop(['year', 'fullName', 'leag', 'gp', 'ilkid'], axis=1).mean()
            near_players_data[near_player] = {
                'first_year': first_year,
                'mean': data_mean
            }
        play_with_ass[id_fullName[play]] = {
            'ass_players': near_players_data,
            'distances': dis
        }
        
    

                  0
minutes        2958
pts            1755
oreb            169
dreb            344
reb             513
asts            197
stl             115
blk              22
turnover        313
pf              234
fg_p       0.451064
ft_p       0.795591
tp_p       0.320896
year           1994
leag              N
gp               80
ilkid     ROBINGL01
                  0
minutes        3077
pts            1772
oreb            219
dreb            224
reb             443
asts            270
stl              89
blk              16
turnover        280
pf              241
fg_p       0.496487
ft_p       0.797101
tp_p       0.227273
year           1981
leag              N
gp               82
ilkid     TRIPUKE01
                  0
minutes        2867
pts            1671
oreb             79
dreb            209
reb             288
asts            194
stl             106
blk              40
turnover        231
pf              219
fg_p       0.463731
ft_p       0.715625
tp_p       0.192308


IndexError: single positional indexer is out-of-bounds

In [126]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(play_with_ass)

{   'borisdiaw': {   'ass_players': {   'jimbrewer': {   'first_year':                   0
minutes        1862
pts             500
oreb            207
dreb            317
reb             524
asts            149
stl              46
blk              35
turnover          0
pf              192
fg_p       0.383212
ft_p       0.650407
tp_p              0
year           1973
leag              N
gp               82
ilkid     BREWEJI01,
                                                         'mean': minutes     1595.181818
pts          400.636364
oreb         165.363636
dreb         283.090909
reb          448.454545
asts         102.272727
stl           56.090909
blk           52.363636
turnover      39.000000
pf           155.454545
fg_p           0.453851
ft_p           0.479582
tp_p           0.015152
dtype: float64},
                                        'kevinrestani': {   'first_year':                   0
minutes        1755
pts             411
oreb            131
dreb            272


    'keithbogans': {   'ass_players': {   'duanecauswell': {   'first_year':                   0
minutes        1719
pts             525
oreb            141
dreb            250
reb             391
asts             69
stl              49
blk             148
turnover         96
pf              225
fg_p       0.508475
ft_p       0.636364
tp_p              0
year           1990
leag              N
gp               76
ilkid     CAUSWDU01,
                                                               'mean': minutes     855.363636
pts         240.727273
oreb         72.181818
dreb        134.454545
reb         206.636364
asts         22.090909
stl          20.000000
blk          69.727273
turnover     45.454545
pf          133.727273
fg_p          0.497257
ft_p          0.580660
tp_p          0.060606
dtype: float64},
                                          'kennythomas': {   'first_year':                   0
minutes        1797
pts             594
oreb            147
dreb            290


In [127]:
len(play_with_ass)

5

In [128]:
mean_for_each_player = {}
for play in play_with_ass:
    ass_v = [ ]
    for ass in play_with_ass[play]:
        for oi in play_with_ass[play]['ass_players']:
            ass_v.append(play_with_ass[play]['ass_players'][oi]['mean'])
    mean_for_each_player[play] = pd.DataFrame(ass_v).mean()

In [129]:
print(play_smaller_dis_name, play_smaller_dis)
print(mean_for_each_player[play_smaller_dis_name])
print(data_name_index.loc[play_smaller_dis_name, :])

borisdiaw 274.9888175989955
minutes     1714.533030
pts          598.150606
oreb         131.602727
dreb         259.521515
reb          391.124242
asts         105.401212
stl           50.098182
blk           55.382727
turnover      74.946667
pf           179.114242
fg_p           0.457174
ft_p           0.650444
tp_p           0.055535
dtype: float64
              ilkid  year leag  gp  minutes  pts  oreb  dreb  reb  asts  stl  \
fullName                                                                       
borisdiaw  DIAWBO01  2003    N  76     1919  342   111   231  342   182   59   
borisdiaw  DIAWBO01  2004    N  66     1201  314    54   116  170   149   37   

           blk  turnover   pf      fg_p      ft_p      tp_p  
fullName                                                     
borisdiaw   37     126.0  191  0.447284  0.602151  0.230769  
borisdiaw   18      87.0  123  0.421769  0.740260  0.180000  


In [133]:
play_with_ass

{'carmeloanthony': {'ass_players': {'glennrobinson': {'first_year':                   0
    minutes        2958
    pts            1755
    oreb            169
    dreb            344
    reb             513
    asts            197
    stl             115
    blk              22
    turnover        313
    pf              234
    fg_p       0.451064
    ft_p       0.795591
    tp_p       0.320896
    year           1994
    leag              N
    gp               80
    ilkid     ROBINGL01, 'mean': minutes     2303.909091
    pts         1293.818182
    oreb          93.363636
    dreb         287.272727
    reb          380.636364
    asts         170.818182
    stl           75.090909
    blk           35.363636
    turnover     195.181818
    pf           167.909091
    fg_p           0.457090
    ft_p           0.828521
    tp_p           0.344844
    dtype: float64}, 'kellytripucka': {'first_year':                   0
    minutes        3077
    pts            1772
    oreb      

In [145]:
for play in play_with_ass:
    print("JOGADOR DE TESTE")
    #print(data_name_index.loc[play, :])
    
    for ass in play_with_ass[play]['ass_players']:
        print("DADOS DOS JOGADORES ASSOCIADOS")
        print(data_name_index.loc[ass, :])
    
        print("------------")
    
    print("***********************")
    print("FINAL DO JOGADORRRR")
    print("***********************")
    

JOGADOR DE TESTE
DADOS DOS JOGADORES ASSOCIADOS
                   ilkid  year leag  gp  minutes   pts  oreb  dreb  reb  asts  \
fullName                                                                        
glennrobinson  ROBINGL01  1994    N  80     2958  1755   169   344  513   197   
glennrobinson  ROBINGL01  1995    N  82     3249  1660   136   368  504   293   
glennrobinson  ROBINGL01  1996    N  80     3114  1689   130   372  502   248   
glennrobinson  ROBINGL01  1997    N  56     2294  1306    82   224  306   158   
glennrobinson  ROBINGL01  1998    N  47     1579   865    73   203  276   100   
glennrobinson  ROBINGL01  1999    N  81     2909  1693   107   378  485   193   
glennrobinson  ROBINGL01  2000    N  76     2813  1674   124   402  526   252   
glennrobinson  ROBINGL01  2001    N  66     2343  1366    70   335  405   168   
glennrobinson  ROBINGL01  2002    N  69     2591  1436    86   371  457   205   
glennrobinson  ROBINGL01  2003    N  42     1336   698    46 

                  ilkid  year leag  gp  minutes  pts  oreb  dreb  reb  asts  \
fullName                                                                      
kevinrestani  RESTAKE01  1974    N  76     1755  411   131   272  403   119   
kevinrestani  RESTAKE01  1975    N  82     1650  492   115   261  376    96   
kevinrestani  RESTAKE01  1976    N  64     1116  358    81   181  262    88   
kevinrestani  RESTAKE01  1977    N   8       84   26     4    10   14     9   
kevinrestani  RESTAKE01  1977    N  54      547  153    36    72  108    30   
kevinrestani  RESTAKE01  1977    N  46      463  127    32    62   94    21   
kevinrestani  RESTAKE01  1978    N  81     1598  575   141   244  385   122   
kevinrestani  RESTAKE01  1979    N  82     1966  874   142   244  386   189   
kevinrestani  RESTAKE01  1980    N  64      999  449    71   103  174    81   
kevinrestani  RESTAKE01  1981    N  13      145   21     8    27   35     7   
kevinrestani  RESTAKE01  1981    N  47      483   74

                ilkid  year leag  gp  minutes   pts  oreb  dreb  reb  asts  \
fullName                                                                     
samperkins  PERKISA01  1984    N  82     2317   903   189   416  605   135   
samperkins  PERKISA01  1985    N  80     2626  1234   195   490  685   153   
samperkins  PERKISA01  1986    N  80     2687  1186   197   419  616   146   
samperkins  PERKISA01  1987    N  75     2499  1066   201   400  601   118   
samperkins  PERKISA01  1988    N  78     2860  1171   235   453  688   127   
samperkins  PERKISA01  1989    N  76     2668  1206   209   363  572   175   
samperkins  PERKISA01  1990    N  73     2504   983   167   371  538   108   
samperkins  PERKISA01  1991    N  63     2332  1041   192   364  556   141   
samperkins  PERKISA01  1992    N  79     2351  1036   163   361  524   156   
samperkins  PERKISA01  1992    N  30      762   363    52    93  145    28   
samperkins  PERKISA01  1992    N  49     1589   673   111   268 

               ilkid  year leag  gp  minutes   pts  oreb  dreb  reb  asts  \
fullName                                                                    
robertreid  REIDRO01  1977    N  80     1849   585   111   248  359   121   
robertreid  REIDRO01  1978    N  82     2259   895   129   354  483   230   
robertreid  REIDRO01  1979    N  76     2304   991   140   301  441   244   
robertreid  REIDRO01  1980    N  82     2963  1301   164   419  583   344   
robertreid  REIDRO01  1981    N  77     2913  1035   175   336  511   314   
robertreid  REIDRO01  1983    N  64     1936   895    97   244  341   217   
robertreid  REIDRO01  1984    N  82     1763   713    81   192  273   171   
robertreid  REIDRO01  1985    N  82     2157   986    67   234  301   222   
robertreid  REIDRO01  1986    N  75     2594  1029    47   242  289   323   
robertreid  REIDRO01  1987    N  62      980   393    38    87  125    67   
robertreid  REIDRO01  1988    N  82     2152  1207    82   220  302   153   

                   ilkid  year leag  gp  minutes  pts  oreb  dreb  reb  asts  \
fullName                                                                       
dudleybradley  BRADLDU01  1979    N  82     2027  688    69   154  223   252   
dudleybradley  BRADLDU01  1980    N  82     1867  657    70   123  193   188   
dudleybradley  BRADLDU01  1981    N  64      937  325    30    57   87    80   
dudleybradley  BRADLDU01  1982    N  58      683  201    27    78  105   106   
dudleybradley  BRADLDU01  1984    N  73     1232  358    34   100  134   173   
dudleybradley  BRADLDU01  1985    N  70      842  195    24    71   95   107   
dudleybradley  BRADLDU01  1986    N  68      900  212    31    71  102    66   
dudleybradley  BRADLDU01  1987    N   2        5    0     0     1    1     1   
dudleybradley  BRADLDU01  1987    N  63     1432  423    25   101  126   150   
dudleybradley  BRADLDU01  1987    N  65     1437  423    25   102  127   151   
dudleybradley  BRADLDU01  1988    N  38 